<a href="https://colab.research.google.com/github/areebaxijaz/AER850-Project-2/blob/main/AER850Project2Part1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1: Steps 1-4 (Data Processing, Model Building, Training, and Evaluation)**

## Step 1: Data Processing: Use ImageDataGenerator from Keras to augment and load the images.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Project 2 Data.zip to Project 2 Data.zip


In [6]:
!unzip "Project 2 Data.zip"

Archive:  Project 2 Data.zip
   creating: Data/
   creating: Data/test/
   creating: Data/test/crack/
  inflating: Data/test/crack/IMG_20230511_101043_jpg.rf.0f754b4a1df6afcfed04bb8468a5f2cb.jpg  
  inflating: Data/test/crack/IMG_20230511_101049_jpg.rf.0cb3a87694d1ef816b1ad557175eb6f8.jpg  
  inflating: Data/test/crack/IMG_20230511_101049_jpg.rf.1f2f7a74cb02d76f7d523d1ae25cb1bf.jpg  
  inflating: Data/test/crack/IMG_20230511_101055_jpg.rf.d6d6f95e4365a1fda10c0ecbd0a3bb5a.jpg  
  inflating: Data/test/crack/IMG_20230511_102305_jpg.rf.ef4f930cc9c6f3d84f00622649d97503.jpg  
  inflating: Data/test/crack/IMG_20230511_102314_jpg.rf.4a4a03ff54b9657b540a073d7602fda7.jpg  
  inflating: Data/test/crack/IMG_20230511_102354_jpg.rf.fcbba70844566e28f4920a3619857542.jpg  
  inflating: Data/test/crack/IMG_20230511_102400_jpg.rf.2d4cf4ae499a7518335c95f2fe4f9bc2.jpg  
  inflating: Data/test/crack/IMG_20230511_103018_jpg.rf.a08632c1cf36bb67517c18010dc1c2c3.jpg  
  inflating: Data/test/crack/IMG_20230511_1

In [7]:
# Importing required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np